In [1]:
from pgmpy.factors.discrete import TabularCPD

In [2]:
from pgmpy.models import BayesianModel

In [3]:
olympic_model = BayesianModel([('Genetics', 'Olympic_Trials'), ('Practice', 'Olympic_Trials'), ('Olympic_Trials', 'Offer')])

In [11]:
genetics_cpd = TabularCPD(
                variable='Genetics',
                variable_card = 2,
                values = [[.2, .8]])

In [14]:
practice_cpd = TabularCPD(
                variable='Practice',
                variable_card = 2,
                values = [[.7, .3]])

In [18]:
offer_cpd = TabularCPD(
                variable='Offer',
                variable_card = 2,
                values = [[.95, .8, .5],
                         [.05, .2, .5]],
                evidence = ['Olympic_Trials'],
                evidence_card = [3])

In [33]:
olympic_trials_cpd = TabularCPD(
                variable='Olympic_Trials',
                variable_card = 3,
                values = [[.5, .8, .8, .9],
                          [.3, .15, .1, .08],
                          [.2, .05, .1, .02]],
                evidence = ['Genetics', 'Practice'],
                evidence_card = [2,2])

In [34]:
olympic_model.add_cpds(genetics_cpd, practice_cpd, offer_cpd, olympic_trials_cpd)

In [35]:
olympic_model.get_cpds()

[<TabularCPD representing P(Genetics:2) at 0x2077ff1b688>,
 <TabularCPD representing P(Practice:2) at 0x2077ff21448>,
 <TabularCPD representing P(Offer:2 | Olympic_Trials:3) at 0x2077ff2adc8>,
 <TabularCPD representing P(Olympic_Trials:3 | Genetics:2, Practice:2) at 0x2077ff687c8>]

In [36]:
olympic_model.active_trail_nodes('Genetics')

{'Genetics': {'Genetics', 'Offer', 'Olympic_Trials'}}

In [37]:
olympic_model.active_trail_nodes('Olympic_Trials')

{'Olympic_Trials': {'Genetics', 'Offer', 'Olympic_Trials', 'Practice'}}

In [38]:
olympic_model.local_independencies('Genetics')

(Genetics _|_ Practice)

In [39]:
olympic_model.local_independencies('Olympic_Trials')

In [40]:
olympic_model.get_independencies()

(Genetics _|_ Practice)
(Genetics _|_ Offer | Olympic_Trials)
(Genetics _|_ Offer | Olympic_Trials, Practice)
(Practice _|_ Genetics)
(Practice _|_ Offer | Olympic_Trials)
(Practice _|_ Offer | Genetics, Olympic_Trials)
(Offer _|_ Genetics, Practice | Olympic_Trials)
(Offer _|_ Practice | Genetics, Olympic_Trials)
(Offer _|_ Genetics | Olympic_Trials, Practice)

In [41]:
from pgmpy.inference import VariableElimination

In [42]:
olympic_infer = VariableElimination(olympic_model)

In [64]:
prob_offer = olympic_infer.query(variables = ['Offer', 'Genetics'])
print(prob_offer)

Eliminating: Practice: 100%|██████████| 2/2 [00:00<00:00, 995.56it/s]


+-------------+----------+-----------------------+
| Genetics    | Offer    |   phi(Genetics,Offer) |
+=============+==========+=======================+
| Genetics(0) | Offer(0) |                0.1684 |
+-------------+----------+-----------------------+
| Genetics(0) | Offer(1) |                0.0316 |
+-------------+----------+-----------------------+
| Genetics(1) | Offer(0) |                0.7214 |
+-------------+----------+-----------------------+
| Genetics(1) | Offer(1) |                0.0786 |
+-------------+----------+-----------------------+


In [63]:
prob_offer_good_genes = olympic_infer.query(variables = ['Offer'], 
                                            evidence = {'Genetics':0})
print (prob_offer_good_genes)

Eliminating: Practice: 100%|██████████| 2/2 [00:00<00:00, 2012.62it/s]


+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.8420 |
+----------+--------------+
| Offer(1) |       0.1580 |
+----------+--------------+


In [66]:
prob_offer_good_genes = olympic_infer.query(variables = ['Offer'], 
                                            evidence = {'Genetics':1, 'Practice':1})
print (prob_offer_good_genes)

Eliminating: Olympic_Trials: 100%|██████████| 1/1 [00:00<00:00, 998.88it/s]


+----------+--------------+
| Offer    |   phi(Offer) |
+==========+==============+
| Offer(0) |       0.9290 |
+----------+--------------+
| Offer(1) |       0.0710 |
+----------+--------------+


In [70]:
prob_good_genes_amazing_olympic_trials = olympic_infer.query(variables = ['Genetics'], 
                                            evidence = {'Olympic_Trials':2, 'Practice':0})
print (prob_good_genes_amazing_olympic_trials)

Eliminating: Offer: 100%|██████████| 1/1 [00:00<00:00, 998.88it/s]


+-------------+-----------------+
| Genetics    |   phi(Genetics) |
+=============+=================+
| Genetics(0) |          0.3333 |
+-------------+-----------------+
| Genetics(1) |          0.6667 |
+-------------+-----------------+


In [72]:
prob_good_genes_no_practice = olympic_infer.query(variables = ['Genetics'], 
                                            evidence = {'Practice':1})
print (prob_good_genes_no_practice)

Eliminating: Offer: 100%|██████████| 2/2 [00:00<00:00, 999.12it/s]


+-------------+-----------------+
| Genetics    |   phi(Genetics) |
+=============+=================+
| Genetics(0) |          0.2000 |
+-------------+-----------------+
| Genetics(1) |          0.8000 |
+-------------+-----------------+


In [73]:
olympic_infer.map_query(variables=['Genetics'])

Eliminating: Practice: 100%|██████████| 3/3 [00:00<00:00, 1498.68it/s]


{'Genetics': 1}

In [74]:
olympic_infer.map_query(variables=['Offer'])

Eliminating: Practice: 100%|██████████| 3/3 [00:00<00:00, 1498.86it/s]


{'Offer': 0}

In [81]:
olympic_infer.map_query(variables=['Olympic_Trials'], evidence = {'Genetics':0, 'Practice':0})

Eliminating: Offer: 100%|██████████| 1/1 [00:00<00:00, 998.64it/s]


{'Olympic_Trials': 0}

In [76]:
help(olympic_infer.map_query)

Help on method map_query in module pgmpy.inference.ExactInference:

map_query(variables=None, evidence=None, elimination_order='MinFill', show_progress=True) method of pgmpy.inference.ExactInference.VariableElimination instance
    Computes the MAP Query over the variables given the evidence.
    
    Note: When multiple variables are passed, it returns the map_query for each
    of them individually.
    
    Parameters
    ----------
    variables: list
        list of variables over which we want to compute the max-marginal.
    evidence: dict
        a dict key, value pair as {var: state_of_var_observed}
        None if no evidence
    elimination_order: list
        order of variable eliminations (if nothing is provided) order is
        computed automatically
    
    Examples
    --------
    >>> from pgmpy.inference import VariableElimination
    >>> from pgmpy.models import BayesianModel
    >>> import numpy as np
    >>> import pandas as pd
    >>> values = pd.DataFrame(np.ra